In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import circuit_drawer
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from PIL import Image  

def preprocess_data(csv_file, sample_size=16):
    """
    Preprocess the CSV data 
    """
    # Read the CSV file
    print(f"Reading CSV file and taking the first {sample_size} samples...")
    df = pd.read_csv(csv_file)
    
    # Take the first `sample_size` rows
    sample_df = df.head(sample_size).reset_index(drop=True)
    
    print("\nSample fraud distribution:")
    print(sample_df['isFraud'].value_counts())
    
    # Make sure there is at least one fraud
    if sample_df['isFraud'].sum() == 0:
        print("WARNING: No fraud transactions found in this sample!")
    
    return sample_df

def create_oracle(n_qubits, marked_state):
    """
    Oracle that marks a single fraudulent transaction state
    """
    oracle = QuantumCircuit(n_qubits)
    
    # Apply X gates to flip the marked state
    for qubit in range(n_qubits):
        if not (marked_state >> qubit) & 1:
            oracle.x(qubit)
    
    # Apply multi-controlled Z gate
    oracle.h(n_qubits-1)
    oracle.mcx(list(range(n_qubits-1)), n_qubits-1)
    oracle.h(n_qubits-1)
    
    # Uncompute the X gates
    for qubit in range(n_qubits):
        if not (marked_state >> qubit) & 1:
            oracle.x(qubit)
    
    oracle.barrier(range(n_qubits))
    return oracle

def create_diffuser(n_qubits):
    """
    Creates the diffusion operator for Grover's algorithm
    """
    diffuser = QuantumCircuit(n_qubits)
    # Apply H gates to all qubits
    for qubit in range(n_qubits):
        diffuser.h(qubit)
    
    # Apply X gates to all qubits
    for qubit in range(n_qubits):
        diffuser.x(qubit)
    
    # Apply multi-controlled Z gate
    diffuser.h(n_qubits-1)
    diffuser.mcx(list(range(n_qubits-1)), n_qubits-1)
    diffuser.h(n_qubits-1)
    
    # Apply X gates to all qubits
    for qubit in range(n_qubits):
        diffuser.x(qubit)
    
    # Apply H gates to all qubits
    for qubit in range(n_qubits):
        diffuser.h(qubit)

    diffuser.barrier(range(n_qubits))
    
    return diffuser

def grover_algorithm(n_qubits, marked_state):
    qc = QuantumCircuit(n_qubits, n_qubits)
    qc.h(range(n_qubits))
    qc.barrier(range(n_qubits))
    qc.compose(create_oracle(n_qubits, marked_state), inplace=True)
    qc.compose(create_diffuser(n_qubits), inplace=True)
    qc.measure(range(n_qubits), range(n_qubits))
    return qc

def plot_results(counts, n_qubits=4, title="Grover's Algorithm for Fraud Detection", filename='grover_histogram.png'):
    """
    Plot histogram of results as percentages and save as an image file.
    """
    # Sort counts (top 10)
    sorted_counts = dict(sorted(counts.items()))

    total_shots = sum(counts.values())
    probabilities = {k: v / total_shots for k, v in sorted_counts.items()}

    # Prepare binary labels padded to n_qubits length
    binary_labels = [format(int(k, 2), f'0{n_qubits}b') for k in sorted_counts.keys()]

    # Plot
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range(len(probabilities)), [p * 100 for p in probabilities.values()])

    plt.title(title, fontsize=14, pad=20)
    plt.xlabel('Quantum State Pattern (Binary)', fontsize=12)
    plt.ylabel('Probability (%)', fontsize=12)
    plt.ylim(0, 100)  # Y-axis from 0% to 100%

    # Add percentage labels on top of each bar
    for bar, prob in zip(bars, probabilities.values()):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{prob:.2%}',
                 ha='center', va='bottom')

    plt.xticks(range(len(binary_labels)), binary_labels, rotation=45)
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()

    # Print results to console
    print("\nQuantum State Patterns, Counts, and Probabilities:")
    for i, (pattern, count) in enumerate(zip(binary_labels, sorted_counts.values()), 1):
        prob = probabilities[pattern]
        print(f"Pattern {i}: {pattern} (Count: {count}, Probability: {prob:.4f})")

    # Save histogram
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Histogram saved as '{filename}'")

def open_image(image_path):
    """
    Opens the generated image using Pillow, with error handling.
    """
    try:
        img = Image.open(image_path)
        img.show()
        print(f"Opened image: {image_path}")
    except Exception as e:
        print(f"Failed to open image '{image_path}': {e}")


def main():
    csv_file = 'Fraud.csv'

    sample_size = 16  # 16 transactions = 4 qubits

    # Step 1: Load sample
    sample_df = preprocess_data(csv_file, sample_size=sample_size)

    # Step 2: Set number of qubits
    n_qubits = int(np.ceil(np.log2(sample_size)))  # Should be 4 for 16 transactions

    # Step 3: Get fraud indices (row numbers where isFraud==1)
    fraudulent_indices = sample_df.index[sample_df['isFraud'] == 1].tolist()
    print(f"\nMarked states (fraud indices): {fraudulent_indices}")

    if not fraudulent_indices:
        print("No fraud found. Please pick another sample or add one manually.")
        return
    
    marked_state = fraudulent_indices[0]
    print(f"Using single marked state: {marked_state}")

    fraud_transaction = sample_df.loc[marked_state]
    print("\n--- Transaction Details of the Marked State ---")
    print(f"Index: {marked_state}")
    print(f"Type: {fraud_transaction['type']}")
    print(f"Amount: {fraud_transaction['amount']}")
    print(f"Old Balance (Origin): {fraud_transaction['oldbalanceOrg']}")
    print(f"New Balance (Origin): {fraud_transaction['newbalanceOrig']}")
    print(f"Is Fraud: {fraud_transaction['isFraud']}")

    # Step 4: Build Grover circuit
    qc = grover_algorithm(n_qubits, marked_state)

    # STEP 5: Save the circuit diagram
    print("\nSaving circuit diagram...")
    circuit_image = circuit_drawer(qc, output='mpl')
    circuit_image.savefig('grover_circuit_ibm.png', dpi=300)
    plt.close()
    print("Circuit diagram saved as 'grover_circuit_ibm.png'")

    # STEP 6: Run the circuit
    simulator = AerSimulator()
    result = simulator.run(qc, shots=1024).result()
    counts = result.get_counts()

    # STEP 7: Plot and open results
    plot_results(counts, n_qubits=n_qubits, title="Grover's Algorithm for Fraud Detection", filename='grover_histogram.png')
    open_image('grover_circuit_ibm.png')
    open_image('grover_histogram.png')

if __name__ == "__main__":
    main()


Reading CSV file and taking the first 16 samples...

Sample fraud distribution:
isFraud
0    13
1     3
Name: count, dtype: int64

Marked states (fraud indices): [2, 3, 9]
Using single marked state: 2

--- Transaction Details of the Marked State ---
Index: 2
Type: TRANSFER
Amount: 181.0
Old Balance (Origin): 181.0
New Balance (Origin): 0.0
Is Fraud: 1

Saving circuit diagram...
Circuit diagram saved as 'grover_circuit_ibm.png'

Quantum State Patterns, Counts, and Probabilities:
Pattern 1: 0000 (Count: 39, Probability: 0.0381)
Pattern 2: 0001 (Count: 27, Probability: 0.0264)
Pattern 3: 0010 (Count: 478, Probability: 0.4668)
Pattern 4: 0011 (Count: 36, Probability: 0.0352)
Pattern 5: 0100 (Count: 38, Probability: 0.0371)
Pattern 6: 0101 (Count: 40, Probability: 0.0391)
Pattern 7: 0110 (Count: 41, Probability: 0.0400)
Pattern 8: 0111 (Count: 32, Probability: 0.0312)
Pattern 9: 1000 (Count: 35, Probability: 0.0342)
Pattern 10: 1001 (Count: 45, Probability: 0.0439)
Pattern 11: 1010 (Count: 